# **Assignment 1: Part B**

Experiment on multivariate linear regression. You are allowed to use all numeric features from the dataset and train a multivariate linear regression model.

### 1. Load and Prepare Datasets

In [1]:
#import packages
import pandas as pd
import numpy as np
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

#Train file URL Link
csv_url_train = "https://raw.githubusercontent.com/shalimarchalhoub/Regression-Assignment/main/cancer_us_county-training.csv"

#Test file URL Link
csv_url_test = "https://raw.githubusercontent.com/shalimarchalhoub/Regression-Assignment/main/cancer_us_county-testing.csv"

#Load Train data into DataFrame df_train
df_train = pd.read_csv(csv_url_train)

#Load Test Data into Datframe df_test
df_test = pd.read_csv(csv_url_test)


### 2. Data Preparationts

Visualizing the Target variable

In [2]:
scatter_chart = alt.Chart(df_train).mark_circle(size=15, color='red').encode(
    x='Id',
    y='TARGET_deathRate'
)
scatter_chart

alt.Chart(...)

Pre processing Train Dataset

In [3]:
#Setting bounds to my data
df_train = df_train[(df_train['TARGET_deathRate'] >= 100) & (df_train['TARGET_deathRate'] <= 240)]

#Removing inconsistent values in avgAnnCOunt
df_train.drop(df_train[(df_train['avgAnnCount'] == 1962.667684)].index, inplace=True)
print(df_train.shape)


(2226, 35)


<ipython-input-3-c9f00dd18bf0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.drop(df_train[(df_train['avgAnnCount'] == 1962.667684)].index, inplace=True)


Pre Processing Test Dataset

In [4]:
df_test = df_test[(df_test['TARGET_deathRate'] >=130) & (df_test['TARGET_deathRate'] <= 240)]
df_test.drop(df_test[(df_test['avgAnnCount'] == 1962.667684)].index, inplace=True)


<ipython-input-4-59e40d2987fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(df_test[(df_test['avgAnnCount'] == 1962.667684)].index, inplace=True)


In [5]:
#Selecting Numeric Columns
numeric_cols = df_train.select_dtypes(include='number').columns
df_train_x = df_train.loc[:, numeric_cols]

#Dropping the target Variable from x_train
df_train_x.drop('TARGET_deathRate',axis=1, inplace=True)


x_train = df_train_x #predictor dataset

#Select the target variable for y_train
y_train = df_train['TARGET_deathRate'] #target variable 


Pre Processing Test Dataset

In [6]:
#Selecting Numeric Columns
numeric_cols_t = df_test.select_dtypes(include='number').columns
df_test_x = df_test.loc[:, numeric_cols_t]
#Dropping the target Variable from x_test
df_test_x.drop('TARGET_deathRate',axis=1, inplace=True)

x_test = df_test_x #predictor dataset

#select the target variable for y_test
y_test = df_test['TARGET_deathRate'].values #target variable 


Check the first 5 rows of x_train as well as the summary of the train dataset

In [7]:
#First 5 rows of x_train
x_train.head()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,...,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Id
1,73.0,35,345.6,29782,21903,38.8,0.000000,32.3,30.8,35.2,...,18.8,45.3,34.1,94.791383,1.649850,0.063631,2.854286,52.818296,4.799131,1
2,292.0,124,468.4,41955,48985,15.5,0.000000,42.2,40.9,43.8,...,44.9,34.5,16.0,95.102348,1.741749,0.376429,0.445611,50.560800,3.996826,2
4,43.0,20,349.7,26309,8551,35.3,0.000000,43.9,41.2,47.8,...,30.4,45.1,24.5,24.535525,73.223736,0.394100,1.396239,33.641208,3.166561,4
5,4139.0,1292,392.9,97279,1040116,7.2,184.594795,38.5,37.0,40.0,...,56.4,23.0,11.5,55.626762,17.607940,14.561938,7.859930,53.702411,5.281995,5
7,26.0,14,334.7,36627,5977,24.1,0.000000,38.2,37.4,38.6,...,28.0,40.3,28.3,89.995006,3.878808,0.000000,3.712336,52.672453,4.238619,7


In [8]:
#Summary of x_train
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2226 entries, 1 to 2437
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              2226 non-null   float64
 1   avgDeathsPerYear         2226 non-null   int64  
 2   incidenceRate            2226 non-null   float64
 3   medIncome                2226 non-null   int64  
 4   popEst2015               2226 non-null   int64  
 5   povertyPercent           2226 non-null   float64
 6   studyPerCap              2226 non-null   float64
 7   MedianAge                2226 non-null   float64
 8   MedianAgeMale            2226 non-null   float64
 9   MedianAgeFemale          2226 non-null   float64
 10  AvgHouseholdSize         2226 non-null   float64
 11  PercentMarried           2226 non-null   float64
 12  PctNoHS18_24             2226 non-null   float64
 13  PctHS18_24               2226 non-null   float64
 14  PctSomeCol18_24         

In [9]:
x_train.describe()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,...,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Id
count,2226.000000,2226.000000,2226.000000,2226.000000,2.226000e+03,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000,...,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000,2226.000000
mean,498.159030,189.386343,447.136613,46891.263252,1.031894e+05,16.957996,153.465026,45.253369,39.577942,42.166173,...,41.126011,36.332120,19.317341,83.372911,9.459261,1.249216,1.964221,51.158102,5.555194,1518.607367
std,1234.166572,450.307438,53.976217,11879.039582,2.814095e+05,6.191429,504.576881,44.918304,5.199514,5.263395,...,9.348258,7.696486,5.902908,16.255195,14.426516,2.627298,3.550081,6.478116,1.918892,875.725652
min,6.000000,3.000000,201.300000,22640.000000,8.270000e+02,3.200000,0.000000,22.300000,22.400000,22.300000,...,13.500000,11.200000,2.700000,10.199155,0.000000,0.000000,0.000000,22.992490,0.000000,1.000000
25%,72.000000,30.000000,416.825000,38647.500000,1.268125e+04,12.300000,0.000000,37.800000,36.400000,39.100000,...,34.500000,31.100000,15.200000,76.210542,0.690922,0.254044,0.301305,47.781595,4.473021,764.250000
50%,158.000000,65.000000,451.700000,44806.000000,2.839450e+04,16.200000,0.000000,40.900000,39.600000,42.400000,...,41.000000,36.600000,19.000000,89.457386,2.580735,0.555014,0.829105,51.530969,5.311398,1507.500000
75%,405.750000,160.000000,482.900000,52284.250000,7.584875e+04,20.500000,94.843543,43.900000,42.400000,45.300000,...,47.500000,41.600000,23.175000,95.248793,11.425267,1.227626,2.149002,55.212917,6.380193,2276.500000
max,24965.000000,9445.000000,1014.200000,125635.000000,5.238216e+06,47.400000,9762.308998,624.000000,64.700000,65.700000,...,70.700000,65.100000,46.600000,100.000000,84.866024,42.619425,41.930251,78.075397,18.556701,3046.000000


Checking the first 5 rows of x_test as well as the summary of it

In [10]:
#First 5 rows of x_test
x_test.head()

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,MedianAgeFemale,...,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Id
0,449.0,154,479.8,51880,104926,18.7,57.183158,30.2,28.5,31.8,...,51.0,24.9,13.1,81.260411,4.154831,10.045737,0.876222,41.071243,4.367123,2553
1,340.0,140,438.5,55472,55423,12.4,0.000000,46.9,44.9,48.2,...,37.6,36.3,16.0,93.660078,0.818115,0.626281,3.116360,57.529142,6.844366,904
2,54.0,18,410.8,49380,10103,11.7,0.000000,49.4,49.4,49.4,...,32.6,40.3,19.4,98.292181,0.041152,0.164609,0.051440,55.928482,1.604585,2192
3,94.0,46,403.8,45979,16708,13.5,598.515681,43.9,42.1,45.0,...,45.1,33.1,13.3,96.090377,1.555569,0.715680,0.378541,48.409405,8.255410,1326
4,2718.0,1065,432.1,51527,726106,20.7,60.597213,33.5,32.4,34.6,...,41.7,37.7,25.8,57.002148,7.093743,14.785464,11.692122,51.852122,6.148433,2394


In [11]:
#Summary of x_test
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531 entries, 0 to 608
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avgAnnCount              531 non-null    float64
 1   avgDeathsPerYear         531 non-null    int64  
 2   incidenceRate            531 non-null    float64
 3   medIncome                531 non-null    int64  
 4   popEst2015               531 non-null    int64  
 5   povertyPercent           531 non-null    float64
 6   studyPerCap              531 non-null    float64
 7   MedianAge                531 non-null    float64
 8   MedianAgeMale            531 non-null    float64
 9   MedianAgeFemale          531 non-null    float64
 10  AvgHouseholdSize         531 non-null    float64
 11  PercentMarried           531 non-null    float64
 12  PctNoHS18_24             531 non-null    float64
 13  PctHS18_24               531 non-null    float64
 14  PctSomeCol18_24          1

Displaying the dimensions of each sets

In [12]:
# Solution
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(2226, 32)
(531, 32)
(2226,)
(531,)


Dealing with Missing values


In [13]:
# Dropping the PctSomeCol18_24 column since more than 3/4 of its values are missing in the train dataset
# Also dropping some other irrelevant columns
x_train.drop(['PctSomeCol18_24', 'PctBlack', 'PctAsian', 'PctWhite'],axis=1, inplace=True)
x_test.drop(['PctSomeCol18_24','PctBlack', 'PctAsian', 'PctWhite'],axis=1, inplace=True)

In [14]:
#Replacing Missing values in PctEmployed16_Over and PctPrivateCoverageAlone with the mean in both train and test
PctEmployed16_Over_mean = x_train['PctEmployed16_Over'].mean()
x_train['PctEmployed16_Over'] = x_train['PctEmployed16_Over'].fillna(PctEmployed16_Over_mean)

PctPrivateCoverageAlone_mean = x_train['PctPrivateCoverageAlone'].mean()
x_train['PctPrivateCoverageAlone'] = x_train['PctPrivateCoverageAlone'].fillna(PctPrivateCoverageAlone_mean)

#For the test dataset
employed_mean_test = x_test['PctEmployed16_Over'].mean()
x_test['PctEmployed16_Over'] = x_test['PctEmployed16_Over'].fillna(employed_mean_test)

Coverage_mean = x_test['PctPrivateCoverageAlone'].mean()
x_test['PctPrivateCoverageAlone'] = x_test['PctPrivateCoverageAlone'].fillna(Coverage_mean)

In [15]:
#Removing the Id columns
x_test.drop("Id",axis=1, inplace=True)
x_train.drop("Id",axis=1, inplace=True)

In [16]:
# Making sure missing values are gone from train dataset
x_train.isna().sum()

avgAnnCount                0
avgDeathsPerYear           0
incidenceRate              0
medIncome                  0
popEst2015                 0
povertyPercent             0
studyPerCap                0
MedianAge                  0
MedianAgeMale              0
MedianAgeFemale            0
AvgHouseholdSize           0
PercentMarried             0
PctNoHS18_24               0
PctHS18_24                 0
PctBachDeg18_24            0
PctHS25_Over               0
PctBachDeg25_Over          0
PctEmployed16_Over         0
PctUnemployed16_Over       0
PctPrivateCoverage         0
PctPrivateCoverageAlone    0
PctEmpPrivCoverage         0
PctPublicCoverage          0
PctPublicCoverageAlone     0
PctOtherRace               0
PctMarriedHouseholds       0
BirthRate                  0
dtype: int64

In [17]:
# Making sure missing values are gone from test dataset
x_test.isna().sum()

avgAnnCount                0
avgDeathsPerYear           0
incidenceRate              0
medIncome                  0
popEst2015                 0
povertyPercent             0
studyPerCap                0
MedianAge                  0
MedianAgeMale              0
MedianAgeFemale            0
AvgHouseholdSize           0
PercentMarried             0
PctNoHS18_24               0
PctHS18_24                 0
PctBachDeg18_24            0
PctHS25_Over               0
PctBachDeg25_Over          0
PctEmployed16_Over         0
PctUnemployed16_Over       0
PctPrivateCoverage         0
PctPrivateCoverageAlone    0
PctEmpPrivCoverage         0
PctPublicCoverage          0
PctPublicCoverageAlone     0
PctOtherRace               0
PctMarriedHouseholds       0
BirthRate                  0
dtype: int64

Splitting the train dataset into training and validation

In [18]:
X_train, X_dev, y_train, y_dev = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

Creating a baseline model

In [19]:
#finding average target value
y_mean = y_train.mean()

#creating a numpy array filled with mean value
y_base = np.full(y_train.shape, y_mean)

print(mse(y_train, y_base))


618.2552274207802


## Training The Regression Model

In [20]:
#Instantiating the LinearRegssion class
model = LinearRegression()

# Fit the model on the training set
model.fit(X_train, y_train)

# Predict the data
y_train_preds = model.predict(X_train)

#Check MSE
print(mse(y_train, y_train_preds,squared= False))

17.206152153191756


Displaying the predictions versus the target line charts on the training set

In [21]:
# Graph of perfect values versus predictions
perfect_test = alt.Chart(pd.DataFrame({'target': y_train, 'preds': y_train})).mark_line(color='green').encode(
    x='target',
    y='preds'
)

pred_chart_test = alt.Chart(pd.DataFrame({'target': y_train, 'preds': y_train_preds})).mark_line().encode(
    x='target',
    y='preds'
  )

pred_chart_test + perfect_test

alt.LayerChart(...)

In [22]:
#testing on validation model
# Predict the data
y_dev_preds = model.predict(X_dev)

#Check MSE
print(mse(y_dev, y_dev_preds))

343.34329104538307


In [23]:
#Given that the regular linear regression gave me the best result I will test it on my test set
y_test_preds = model.predict(x_test)

#Check MSE
print(mse(y_test, y_test_preds, squared = False))
print(mse(y_test, y_test_preds))

17.518791955063612
306.90807156480156


In [24]:
# Graph of perfect values versus predictions for test data
perfect_test_t = alt.Chart(pd.DataFrame({'target': y_test, 'preds': y_test})).mark_line(color='green').encode(
    x='target',
    y='preds'
)

pred_chart_t = alt.Chart(pd.DataFrame({'target': y_test, 'preds': y_test_preds})).mark_line().encode(
    x='target',
    y='preds'
  )

pred_chart_t + perfect_test_t

alt.LayerChart(...)